In [9]:
import pandas as pd
pd.set_option('display.max_columns', None)

df = pd.read_parquet('../data/processed/player_stats_Primera_Division_Argentina_2024.parquet')
df.head(10)


,Player,Nation,Pos,Squad,Age,Born,MatchesPlayed,Gls,Ast,xG,xAG,Shots,SoT,PassCmp,PassAtt,PassCmpPct,Tkl,TklW,Blocks,Int,GK_GA,GK_GA90,GK_SoTA,GK_Saves,GK_SavePct,GK_CS,GK_CSPct,GK_PKAtt,GK_PKA,GK_PKsv,GK_PKm,GK_PSxG,GK_PSxG_per_SoT,GK_PSxG_PlusMinus,GK_PSxG_PlusMinus_per90,GK_PassCmp,GK_PassAtt,GK_PassCmpPct,GK_GKPassAtt,GK_Throws,GK_LaunchPct,GK_AvgLen,GK_CrossesStp,GK_CrossesStpPct,GK_OPA,GK_OPA90,GK_OPA_AvgDist,IsGK,AgeYears
0,aaron anselmino,ar ARG,DF,Boca Juniors,18,2005,9.7,0,3,0.0,1.8,0,0,455,527,86.34,26,17,18,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,18.0
1,aaron nicolas molinas,ar ARG,"MF,FW",Defensa y Just,23,2000,17.4,5,4,4.2,2.5,36,21,657,848,77.48,21,11,19,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,23.0
2,aaron quiros,ar ARG,DF,Banfield,22,2001,5.0,0,0,0.1,0.0,2,1,128,182,70.33,3,2,5,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,22.0
3,abiel osorio,ar ARG,FW,Defensa y Just,21,2002,10.7,4,0,3.6,0.8,30,11,171,220,77.73,8,6,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,21.0
4,adam bareiro,py PAR,FW,San Lorenzo,27,1996,2.5,0,0,0.6,0.3,3,1,30,43,69.77,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,27.0
5,adrian balboa,uy URU,FW,Unión,30,1994,23.4,5,2,6.6,1.8,66,19,257,403,63.77,22,16,20,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,30.0
6,adrian guillermo sanchez,ar ARG,MF,Atlé Tucumán,24,1999,24.9,0,0,0.5,1.2,14,1,933,1193,78.21,77,42,42,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,24.0
7,adrian martinez,ar ARG,FW,Racing Club,31,1992,15.9,7,3,9.2,2.1,48,19,145,233,62.23,11,7,6,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,31.0
8,adrian sporle,ar ARG,DF,Independiente,28,1995,12.1,0,0,0.3,0.8,6,4,449,637,70.49,26,18,12,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,28.0
9,agustin almendra,ar ARG,MF,Racing Club,23,2000,13.5,2,2,1.3,0.9,22,8,486,621,78.26,49,29,20,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,23.0


In [11]:
import numpy as np

# checks clave
print(df.shape)
print("Duplicados (Player,Squad):", df.duplicated(subset=["Player","Squad"]).sum())

for a,b,label in [("SoT","Shots","SoT<=Shots"),("PassCmp","PassAtt","PassCmp<=PassAtt")]:
    if {a,b}.issubset(df.columns):
        print(label, "violaciones:", int((df[a] > df[b]).sum()))

# GK en no-arqueros deben ser NaN
gk_cols = [c for c in df.columns if c.startswith("GK_")]
if gk_cols:
    is_gk = df["IsGK"] if "IsGK" in df.columns else df["Pos"].str.contains("GK", na=False)
    print("GK no-nulos en NO arqueros:", int(df.loc[~is_gk, gk_cols].notna().sum().sum()))


(925, 49)
Duplicados (Player,Squad): 0
SoT<=Shots violaciones: 0
PassCmp<=PassAtt violaciones: 0
GK no-nulos en NO arqueros: 0
